In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
# 3party
%config IPCompleter.greedy=True 
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import LeavePGroupsOut
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ParameterGrid
from sklearn.externals import joblib
import matplotlib.pyplot as plt
import numpy as np

# python
import os
from IPython.core.debugger import Tracer;

# classification packages-------------
from supervised_new import *
from prep_input_supervised import *
from plot_classification import *
from plot_statistics import *
from supervised_new import *
#from parameter_search import *
from cross_validation import *
from save_for_class import *
from pipelines import *
from statistics import *
# from plot_r_classification import *


ImportError: No module named supervised_new

In [ ]:

path_to_save = '/Volumes/ASSD/pre_epi_seizures/h5_files/classification_datasets'
class_metadata = ['labels', 'group', 'sample_domain']

In [ ]:
def get_data_struct(pt_list):
    # Select Patient to analyze
    # pt = 3
    global path_to_save
    path_to_save += '/patient_' + str(pt_list)
    
    
    data_struct = load_feature_groups_baseline_seizure_per_patient(
                                                    pt_list,
                                                    'hrv_time_features#'
                                                    )
    
    return data_struct

In [ ]:
def make_dir(dir_):
    if not os.path.exists(dir_):
        print 'this '
        print 'MAKING'
        print dir_
        os.makedirs(dir_)
        print 'second this'


In [ ]:
def general(file_to_save, interval, data_struct, trial):
    
    # Prep data_struct for classification --conversion to pandas DataFrame
    data_struct = prep_input_supervised_baseline_seizure(data_struct, interval).reset_index()
    data_struct = data_struct.drop(['index'], axis=1)

    # Get points per label
    points_per_label = data_struct['labels'].reset_index().groupby(['labels']).count()
    points_per_label = list(dict(points_per_label)['index'])

    # Select features from the data_struct
    feature_names = [name
                     for name in data_struct.columns
                     if name not in class_metadata]

    
    # **************************Dataset Exploratory Statistics**********
    # Run relevant statistics
    univariate_statistics(full_path,
                           X,y, groups,
                           pipe,
                           param_grid, scoring,
                           compute_all_new, cv_out, cv_in)
    path_BOX = full_path + 'BOX/'
    make_dir(path_BOX)
    plot_box_univariate_per_label(path_BOX,
                                  X, y)
    
    stop
    #************************************************************************

    # ***********************************Learning****************************
    # Learn from data_struct using nested cross_validation
    # learninig is an optimization and respective test results
    # for each partition of the dataset according to cv_out
    learning_results = nested_cross_validation(full_path,
                                           X,y, groups,
                                           pipe,
                                           param_grid, scoring,
                                           compute_all_new, cv_out, cv_in)
    #************************************************************************
       
    # Plot reciever operator characteristics
    #path_ROC = full_path + '/ROC/'
    #make_dir(path_ROC)
    #plot_roc_new(path_ROC, interval, points_per_label, learning_results, trial)
   
    # Plot Feature frequecy per label distribution histogram 
    path_HIST = full_path + '/hist/'
    make_dir(path_HIST)
    plot_hist(path_HIST, interval, points_per_label, data_struct, class_metadata)
    
    # Plot Scatter plot of each pair features

In [ ]:
def main():
    ''' Main function'''
    data_struct = get_data_struct(pt_list = [3, 5])

    mode = 'patient_semi_specific'

    file_to_save = '/mode_' + str(mode) 

    for step_min in xrange(5, 50, 5):

         path_step = file_to_save + '/step_' + str(step_min) + '/'

         for up, down in zip(xrange(step_min, 50, step_min), xrange(0, 50 - step_min, step_min)):
            interval = [(down*60, up*60)]

            full_path = path_step + 'parameter_optimization' + '_' + str(interval) + '/'

            trial_list = xrange(0, 1)

            for trial in trial_list:
                general(full_path, interval, data_struct, trial)

In [ ]:
main()
